# 3.5.1. KMeans

## Preparación del Entorno

### Carga de Módulos

In [ ]:
import math
import os
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import offsetbox
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.figure_factory as ff
import session_info
from time import time
from plotly.subplots import make_subplots
from sklearn import set_config
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Tema Principal
from yellowbrick.cluster import SilhouetteVisualizer
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans
import sys

sys.path.append('../scripts')
from funny_stuffs import score_plot, calculate_elbow_point, plot_elbow_method

In [ ]:
session_info.show()

### Configuración Inicial

In [ ]:
random_seed = 333  # Semilla para reproducibilidad de resultados
np.random.seed(random_seed)  # Para reproducibilidad

# Configuración de opciones de visualización para pandas
pd.set_option('display.max_columns', None)  # Muestra todas las columnas
pd.set_option('display.max_rows', 15)  # Ajusta el número de filas a mostrar

# Configuraciones extras
sns.set_style('dark')
dark_template = pio.templates['plotly_dark'].to_plotly_json()
dark_template['layout']['paper_bgcolor'] = 'rgba(30, 30, 30, 0.5)'
dark_template['layout']['plot_bgcolor'] = 'rgba(30, 30, 30, 0.5)'
pio.templates['plotly_dark_semi_transparent'] = go.layout.Template(dark_template)
pio.templates.default = 'plotly_dark_semi_transparent'
set_config(transform_output="pandas")
set_config(display='diagram')
warnings.filterwarnings("ignore")
%matplotlib inline

# Configuración para intentar aplicar un estilo oscuro
plt.rcParams['figure.facecolor'] = 'black'  # Color de fondo de la figura
plt.rcParams['axes.facecolor'] = 'black'  # Color de fondo de los ejes
plt.rcParams['axes.edgecolor'] = 'white'  # Color de los bordes de los ejes
plt.rcParams['axes.labelcolor'] = 'white'  # Color de las etiquetas de los ejes
plt.rcParams['xtick.color'] = 'white'  # Color de las marcas de x
plt.rcParams['ytick.color'] = 'white'  # Color de las marcas de y
plt.rcParams['text.color'] = 'white'  # Color del texto

# Funciones Extras
def calculate_elbow_point(distortions):
   n_points = len(distortions)
   all_coords = np.vstack((range(n_points), distortions)).T
   first_point = all_coords[0]
   line_vec = all_coords[-1] - all_coords[0]
   line_vec_norm = line_vec / np.sqrt(np.sum(line_vec**2))
   vec_from_first = all_coords - first_point
   scalar_product = np.sum(vec_from_first * np.tile(line_vec_norm, (n_points, 1)), axis=1)
   vec_from_first_parallel = np.outer(scalar_product, line_vec_norm)
   vec_to_line = vec_from_first - vec_from_first_parallel
   dist_to_line = np.sqrt(np.sum(vec_to_line ** 2, axis=1))
   return dist_to_line.argmax() + 2

## KMeans

### Fundamento Teórico

El K-Means $K$-Medias (*K*-Means en inglés) es un algoritmo de agrupamiento que divide un conjunto de datos en K grupos o clusters distintos, basándose en las características de los datos. El *objetivo* del algoritmo K-Means es minimizar la suma de distancias cuadráticas entre los puntos y el centroide de su respectivo grupo o cluster.

#### Planteamiento:

Dado conjunto de observaciones $\{x_1, x_2, \ldots, x_n\}$, donde cada observación es un vector real de $d$ dimensiones, $K$-Means busca particionar las $n$ observaciones en $k (\leq n)$ conjuntos $C = \{C_1, C_2, \ldots, C_k\}$ que cumplen:

   1. $C_1 \cup C_2 \cup \ldots \cup C_K = \{x_1, x_2, \ldots, x_n\}$. En otras palabras, cada observación pertenece al menos a uno de los $K$ clusters.
   2. $C_k \cap C_{k'} = \emptyset$ para todo $k \neq k'$. Es decir, los clusters no se solapan: ninguna observación pertenece a más de un cluster.


De manera que se minimice la varianza dentro de cada grupo. Esto se puede expresar como minimizar la siguiente función objetivo (denominada como distorsión):

$$
\min_{C_1,\ldots,C_K} \left\{\frac{1}{|C|} \sum_{i, i' \in C_k} \sum_{x \in C_k} \| x_i - \mu_k \|^2 \right\}.
$$

donde $K$ es el número de clusters, $C_k$ es el conjunto de puntos en el $k$-ésimo cluster, $\mu_k$ es el centroide del $k$-ésimo cluster, y $\| x_i - \mu_k \|^2$ es el cuadrado de la distancia euclidiana entre el punto $x_i$ y el centroide $\mu_k$.

Para lograr minimizar esta función, el siguiente proceso se lleva a cabo:

##### 1. **Inicio**: 

Se eligen $k$ puntos como centroides iniciales de los clusters. Esto puede hacerse al azar o mediante algún criterio específico.

Pensemos en los siguientes puntos:

$$
\begin{bmatrix}
1 & 1 \\
1.5 & 2 \\
3 & 4 \\
5 & 7 \\
3.5 & 5 \\
4.5 & 5 \\
3.5 & 4.5 \\
6 & 8 \\
\end{bmatrix}
$$

Buscamos agruparlos en $k=2$ clusters.

Supongamos que inicialmente seleccionamos dos puntos como centroides iniciales de forma aleatoria. Para este ejemplo, vamos a elegir $P_1$ y $P_4$ como los centroides iniciales. Entonces:

- Centroide inicial $C_1 = P_1 = (1, 1)$
- Centroide inicial $C_2 = P_4 = (5, 7)$

In [ ]:
data_points = np.array([
                        [1, 1],
                        [1.5, 2],
                        [3, 4],
                        [5, 7],
                        [3.5, 5],
                        [4.5, 5],
                        [3.5, 4.5],
                        [6, 8]       ])

In [ ]:
#Inicio:

fig = go.Figure()
fig.add_trace(go.Scatter(x=data_points[:, 0], y=data_points[:, 1], mode='markers', name='Datos'))

centroids = np.array([[1, 1], [5, 7]])
fig.add_trace(go.Scatter(x=centroids[:, 0], y=centroids[:, 1], mode='markers', 
                        marker=dict(color='red', size=12, line=dict(color='DarkRed', width=2)), 
                        name='Centroides Iniciales'))

fig.update_layout(title='Inicio del Proceso:', xaxis_title='X', yaxis_title='Y', legend_title='Leyenda',
                  height=500, width=1200)

fig.show()


##### 2. **Asignación**:

Cada punto del conjunto de datos se asigna al cluster cuyo centroide es el más cercano. Formalmente, esto significa que cada punto $x$ se asigna a un cluster $C_i$ tal que:

   $$
   \min_{\mu_i \in C} \| x - \mu_i \|^2
   $$
   
   donde $C$ es el conjunto de centroides.


Calculamos la distancia de cada punto a los dos centroides y asignamos cada punto al cluster del centroide más cercano. Usaremos la distancia euclidiana para medir la cercanía:

In [ ]:
#Realizamos la siguiente función
def assign_to_clusters(points, centroid_1, centroid_2):
   cluster_assignments = []
   for point in points:
      distance_to_centroid_1 = np.linalg.norm(point - centroid_1)
      distance_to_centroid_2 = np.linalg.norm(point - centroid_2)
      # Asignar al cluster 1 si el punto está más cerca de centroid_1, de lo contrario asignar a cluster 2
      cluster_assignments.append(1 if distance_to_centroid_1 < distance_to_centroid_2 else 2)
   return cluster_assignments

#Procedemos a calcular:
assign_to_clusters(points=data_points,
                  centroid_1= centroids[0],
                  centroid_2= centroids[1])

Después de realizar la asignación de cada punto a los clusters basados en los centroides iniciales $C_1 = (1, 1)$ y $C_2 = (5, 7)$, tenemos:

- $P_1$ y $P_2$ se asignan al Cluster 1.
- $P_3$, $P_4$, $P_5$, $P_6$, $P_7$, y $P_8$ se asignan al Cluster 2.

In [ ]:
# Asignación
cluster_assignments = np.array([1, 1, 2, 2, 2, 2, 2, 2])

fig = go.Figure()

# Cluster 1
cluster_1_points = data_points[cluster_assignments == 1]
fig.add_trace(go.Scatter(x=cluster_1_points[:, 0], y=cluster_1_points[:, 1], mode='markers', marker=dict(color='blue'), name='Cluster 1'))

# Cluster 2
cluster_2_points = data_points[cluster_assignments == 2]
fig.add_trace(go.Scatter(x=cluster_2_points[:, 0], y=cluster_2_points[:, 1], mode='markers', marker=dict(color='green'), name='Cluster 2'))


fig.add_trace(go.Scatter(x=centroids[:, 0], y=centroids[:, 1], mode='markers', marker=dict(color='red', size=12, line=dict(color='DarkRed', width=2)), 
                        name='Centroides'))
fig.update_layout(title='Asignación de Puntos a Clusters', xaxis_title='X', yaxis_title='Y', 
                  height=500, width=1200)

fig.show()


##### 3. **Actualización de centroides**:

Se recalculan los centroides de cada cluster como el promedio de todos los puntos asignados a ese cluster:

   $$
   \mu_i = \frac{1}{|C_i|} \sum_{x \in C_i} x
   $$

In [ ]:
# Separar los puntos basado en las asignaciones de cluster actuales
cluster_1_points = data_points[np.array(cluster_assignments) == 1]
cluster_2_points = data_points[np.array(cluster_assignments) == 2]

# Calcular los nuevos centroides basado en las asignaciones actuales
new_centroid_1 = np.mean(cluster_1_points, axis=0)
new_centroid_2 = np.mean(cluster_2_points, axis=0)

new_centroid_1, new_centroid_2


Siguiendo con la actualización de los centroides, obtenemos los siguientes nuevos centroides para cada cluster: <span style="font-size: xx-small; color: red;">N1</span>

- Nuevo centroide para el Cluster 1: $C_1 = (1.25, 1.5)$
- Nuevo centroide para el Cluster 2: $C_2 = (4.25, 5.58)$

In [ ]:
#Actualización:

fig = go.Figure()
fig.add_trace(go.Scatter(x=data_points[:, 0], y=data_points[:, 1], mode='markers', name='Datos'))

old_centroids = np.array([[1, 1], [5, 7]])
new_centroids = np.array([[1.25, 1.5], [4.25, 5.58]])
fig.add_trace(go.Scatter(x=old_centroids[:, 0], y=old_centroids[:, 1], mode='markers', marker=dict(color='red', size=12, 
                        line=dict(color='DarkRed', width=2)), name='Centroides Iniciales'))
fig.add_trace(go.Scatter(x=new_centroids[:, 0], y=new_centroids[:, 1], mode='markers', marker=dict(color='green', size=12,
                        line=dict(color='DarkGreen', width=2)), name='Centroides Actualizados'))

fig.update_layout(title='Actualización:', xaxis_title='X', yaxis_title='Y', legend_title='Leyenda',
                  height=500, width=1200)

fig.show()

##### 4. **Repetición y Convergencia**:

Los pasos 2 y 3 se repiten hasta que los centroides ya no cambien significativamente o hasta alcanzar un número máximo de iteraciones. Esto indica que se ha alcanzado la convergencia.

In [ ]:
# Convergencia:
kmeans = KMeans(n_clusters=2, init=np.array([[1, 1], [5, 7]]))
kmeans.fit(data_points)

final_centroids = kmeans.cluster_centers_
final_assignments = kmeans.labels_

fig = go.Figure()
for cluster_id in np.unique(final_assignments):
   points = data_points[final_assignments == cluster_id]
   fig.add_trace(go.Scatter(x=points[:, 0], y=points[:, 1], mode='markers',
                           name=f'Cluster {cluster_id+1}'))

fig.add_trace(go.Scatter(x=final_centroids[:, 0], y=final_centroids[:, 1], mode='markers',
                        marker=dict(color='red', size=12, line=dict(color='DarkRed', width=2)), name='Centroides Finales'))

fig.update_layout(title='Asignación Final:', xaxis_title='X', yaxis_title='Y', legend_title='Leyenda',
                  height=500, width=1200)

fig.show()


In [ ]:
kmeans.n_iter_

In [ ]:
# Distorsión:
print("Distorsión:",kmeans.inertia_)
print("Prom. Distancias:",np.sqrt(kmeans.inertia_))

#### Aplicaciónes y Consideraciones

<p style="font-size:25px;">Aplicaciones</p>

1. Biología Computacional y Genómica
2. Análisis de Redes Sociales
3. Segmentación de Clientes
4. Análisis de Documentos y Textos
5. Recomendadores


<p style="font-size:25px;">Ventajas</p>

1. Eficiencia
2. Facilidad de Implementación
3. Adaptable a Nuevos Ejemplos
4. Flexible a Transformaciones


<p style="font-size:25px;">Consideraciones</p>

1. Elección de K
2. Sensibilidad a los centroides iniciales `(k-means++, random)`
3. Sensibilidad a la forma de los clusters
4. Escala de los datos
5. Atrapado en mínimos locales
6. Sensibilidad a los datos atípicos
7. Velocidad y escalabilidad
8. Interpretación
9. Resultados diferentes cada ejecución
10. Evaluación del modelo

### Ejemplo Práctico

#### Preparación de los Datos

**Objetivo**

El objetivo es clasificar las especies de flores de iris en grupos naturales basándonos en características morfológicas de las flores como longitud y anchura de pétalos y sépalos, para entender mejor las relaciones entre diferentes especies y facilitar su identificación y estudio en un contexto no supervisado.

**Planteamiento del problema**

Se ha acumulado un conjunto de datos con 50 muestras de flores de iris recogidas de diferentes hábitats y condiciones geográficas. El equipo de investigación necesita una forma de categorizar las muestras eficientemente para apoyar estudios evolutivos y ecológicos subsecuentes. Sin etiquetas preexistentes para las especies, necesitan un método de agrupamiento que organice las flores en grupos significativos basados en sus características físicas.

Las características extraídas de cada elemento son:

1. **Longitud del Sépalo (Sepal Length)**: La longitud del sépalo en centímetros. El sépalo es la parte que protege el capullo de la flor y a menudo soporta el pétalo cuando está en flor.

2. **Ancho del Sépalo (Sepal Width)**: El ancho del sépalo en centímetros. Esta medida se toma en su parte más ancha.

3. **Longitud del Pétalo (Petal Length)**: La longitud del pétalo en centímetros. Los pétalos son las partes coloreadas y a menudo vistosas de la flor que siguen al sépalo.

4. **Ancho del Pétalo (Petal Width)**: El ancho del pétalo en centímetros. Esta medida se toma en su parte más ancha.

<div style="text-align:center">
  <img src="../docs/figures/irisrm.png" alt="iris">
</div>

**Iris Setosa, Virginica y Versicolor**

In [ ]:
# Carga y diccionario del conjunto de datos
iris = load_iris()
X = pd.DataFrame(iris.data,columns=iris.feature_names)
y = iris.target #No conocido

In [ ]:
X.sample(5)

In [ ]:
X.describe()

Como no hay diferencia de escala no hay necesidad de estandarizar o escalar.

#### Implementación del Método

Principales parámetros de $K$-Means:
```python
KMeans(
    n_clusters=8,
    init='k-means++',
    max_iter=300,
    tol=0.0001,
    random_state=None
)

#### Visualización e Implementación de Resultados

In [ ]:
distortions = []; silhouette_scores = []
davies_bouldin_scores = []; calinski_harabasz_scores = []
range_n_clusters = range(2, 5)  # Ajustamos para explorar desde 1 hasta 4 clusters

for n_clusters in range_n_clusters:
   km = KMeans(n_clusters=n_clusters, init='k-means++', n_init=10, max_iter=300, random_state=random_seed)
   cluster_labels = km.fit_predict(X); distortions.append(km.inertia_)
   silhouette_ = silhouette_score(X, cluster_labels); silhouette_scores.append(silhouette_)
   davies_bouldin_ = davies_bouldin_score(X, cluster_labels); davies_bouldin_scores.append(davies_bouldin_)
   calinski_harabasz_ = calinski_harabasz_score(X, cluster_labels); calinski_harabasz_scores.append(calinski_harabasz_)

In [ ]:
optimal_k = calculate_elbow_point(distortions)
plot_elbow_method(range_n_clusters, distortions, optimal_k)

In [ ]:
score_plot(scores=silhouette_scores, range_n_clusters=range_n_clusters, 
         operator='max', name_index = 'Índice de Silueta')

In [ ]:
score_plot(scores=calinski_harabasz_scores, range_n_clusters=range_n_clusters, 
         operator='max', name_index = 'Índice de C-H')

In [ ]:
for i in range(2, 4):
    km = KMeans(n_clusters=i, init='k-means++', random_state=random_seed)
    visualizer = SilhouetteVisualizer(km, colors='yellowbrick')
    visualizer.fit(X)
    visualizer.show()

In [ ]:
# Generamos KMeans con 2 Clusters
km = KMeans(n_clusters=3, init='k-means++', max_iter=300, random_state=random_seed)
km.fit(X)

In [ ]:
X['Grupos Reales'] = y
X['Grupos K-Means'] = km.labels_

In [ ]:
iris.target_names

In [ ]:
X['Grupos Reales'].values

In [ ]:
X['Grupos K-Means'].values

In [ ]:
X['Grupos Reales'] = X['Grupos Reales'].replace({0: 'G1', 1: 'G2', 2: 'G3'})
X['Grupos K-Means'] = X['Grupos K-Means'].replace({1: 'G1', 0: 'G2', 2: 'G3'})

In [ ]:
X.drop(['Grupos Reales'], axis=1).groupby('Grupos K-Means').mean().round(2)

In [ ]:
X.drop(['Grupos K-Means'], axis=1).groupby('Grupos Reales').mean().round(2)

In [ ]:
confusion_matrix = pd.crosstab(X['Grupos Reales'], X['Grupos K-Means'])

# Matriz de Confusión
fig = ff.create_annotated_heatmap(z=confusion_matrix.values,
                                 x=list(confusion_matrix.columns),
                                 y=list(confusion_matrix.index),
                                 colorscale='Greys')

fig.update_layout(title='Matriz de Confusión',
                  xaxis=dict(title='Grupos K-Means'),
                  yaxis=dict(title='Grupos Reales'))

# Mostrar la figura
fig.show()

Reducción a Dos dimensiones con PCA:

In [ ]:
# Reducción de dimensiones con PCA a 2 componentes
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X.iloc[:,:4])

In [ ]:
# Visualización
df_original = pd.DataFrame({
   'PCA1': X_pca.values[:, 0],
   'PCA2': X_pca.values[:, 1],
   'Species': X['Grupos Reales']
})

df_kmeans = pd.DataFrame({
   'PCA1': X_pca.values[:, 0],
   'PCA2': X_pca.values[:, 1],
   'Cluster': X['Grupos K-Means']
})


df_original['Type'] = 'Original'
df_kmeans['Type'] = 'K-Means'
df_combined = pd.concat([df_original, df_kmeans])
df_combined['Label'] = df_combined.apply(lambda row: row['Species'] if row['Type'] == 'Original' else row['Cluster'], axis=1)

fig = px.scatter(df_combined, x='PCA1', y='PCA2', color='Label', 
                        facet_col='Type', title='Iris Data: Original vs. K-Means Clustering with PCA')

fig.update_layout(height=500, width=1200)

fig.show()


In [ ]:
iris.target_names

**Conclusión**

Aunque hay limitaciones en la separación perfecta de todos los grupos, esta metodología ofrece insights valiosos para la identificación y el análisis de grupos en datos sin etiquetar. El análisis destaca la importancia de combinar técnicas de clustering con análisis visual y conocimiento del dominio para interpretar y etiquetar correctamente los grupos en aplicaciones no supervisadas.